In [4]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime as dt
import pandas as pd
from dateutil import parser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
(parser.parse('2018-01-01 14:23:32.258') - parser.parse('2018-01-01 14:22:23.420'))

datetime.timedelta(0, 68, 838000)

In [25]:
df = pd.DataFrame({
    'date': [dt.datetime(2018, 1, 1), dt.datetime(2018, 1, 2), dt.datetime(2018, 1, 3), dt.datetime(2018, 1, 4), dt.datetime(2018, 1, 5), dt.datetime(2018, 1, 6)],
    'id': [1, 2, 1, 2, 1, 2],
})


def split_by_time_window(df_in, start_cohort, end_cohort, id_col, date_col, obs_size, target_size):
    start = parser.parse(start_cohort)
    end = parser.parse(end_cohort)
    df_in = df_in.sort_values(date_col)
    df = df_in[df_in[date_col] > start].reset_index(drop=True)
    delta = df.groupby(id_col)[date_col].apply(lambda s: (s - s.shift(1)).fillna(0).cumsum())
    df['age'] = delta.dt.days
    tr_idx = (df[date_col] < end) & (df.age <= obs_size)
    ts_idx = (df.age > obs_size) & (df.age <= obs_size+target_size)
    return df[tr_idx], df[ts_idx]


x, y = split_by_time_window(df, '2018-01-01', 'id', 'date', obs_size=2, target_size=2)
x.head()
y.head()

1          NaT
3   2018-01-03
Name: 1, dtype: datetime64[ns]
0          NaT
2   2018-01-02
4   2018-01-04
Name: 2, dtype: datetime64[ns]


,date,id
0,2018-01-02,2
1,2018-01-03,1
2,2018-01-04,2
3,2018-01-05,1


,date,id
4,2018-01-06,2
